# **Seminar - Functional 2P imaging analysis**

## Important Jupyter Notebook commands

**Execute a command field - 'Shift' + 'Enter'**  
**Create a new command field above the selected field - 'Esc' + 'A'**  
**Create a new command field below the selected field - 'Esc' + 'B'**  
**Delete selected command field - 'Esc' + 2x'D'**

## Import of modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec as gridspec
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.cluster import SpectralClustering
from sklearn import decomposition
from random import randrange

from kmeans_euclidean import KMeansEuclidean
from kmeans_mahalanobis import KMeansMahalanobis

In [ ]:

# Install a pip package in the current Jupyter kernel
#import sys
#!{sys.executable} -m pip install numpy

## Support functions

In [ ]:
def plot_heatmap(Cell, df, save, save_path, cmap_c):
    firing = df.pivot_table(Cell, 'Lap_2', '2_cm_binned_position')
    stamps = df.pivot_table(values=Cell, columns='2_cm_binned_position', aggfunc='sum')
    avg_flour = df.pivot_table(values=Cell, columns='2_cm_binned_position', aggfunc='mean')
    fig, axes = plt.subplots(2, 1, figsize=(16,10), 
                             gridspec_kw={'height_ratios': [5, 1]});
    sns.heatmap(firing, vmin=df[Cell].min(), vmax=df[Cell].max(), ax=axes[0], cmap=cmap_c);
    sns.heatmap(avg_flour, annot=False, ax=axes[1], vmin=np.min(avg_flour.values), vmax=np.max(avg_flour.values), cmap=cmap_c);
    if save == True:
        fig.savefig(save_path, format='png')
    #visualization.plot_contours(spatial_filtered[:, Cell], templates[-1]);

## **First steps - Import and inspect data**
### Dataset background: One mouse on a linear treadmill, recorded under same task-settings on consecutive days. At one specified position the animal receives an automatic reward. Data is organized as a DataFrame.

In [ ]:
# Dataset #1
df_day1 = pd.read_excel('DataFrame_example_day1.xlsx')

In [ ]:
# Dataset #2
df_day2 = pd.read_excel('DataFrame_example_day2.xlsx')

### Let's look at the dimensions of our data frame.

In [ ]:
df_day1.shape

### We print the first 50 rows of the data frame.

In [ ]:
df_day1.head(50)

### **Task:** What information can we derive from this? Can you explain what is going on here?

### **Solution:**  

#### This is a bit hard to oversee and it becomes obvious, that we have to clean the data first. We can save the spiking data (actually: spiking probability) as a separate file 'neural_data'. In the data frame there is spiking data from 207 different neurons. There are also 32 frames at the beginning and at the end of the recording which are  empty, so we have to get rid of those.

In [ ]:
df_neural_data_day1 = df_day1.iloc[:, 15:-1]
df_neural_data_day1

#### The rest of the data (e.g. position and velocity information) are storend in a different data frame.

In [ ]:
df_position_data_day1 = df_day1.iloc[:, :15]
df_position_data_day1

### Overview of the columns.

In [ ]:
df_position_data_day1.keys()

### Now we can visualize some of the data like for example the velocity over time:

In [ ]:
df_position_data_day1['Velocity'].plot()

### Or the position over time:

In [ ]:
df_position_data_day1['Position'].plot()

### We can also plot the averaged velocity over position over all laps:

In [ ]:
# Define which data frame is used - in this case we use the data frame that contains the position information
df = df_position_data_day1 

# We aligne the data - in this case we calculate the average velocity for each 2 cm bin
Velocity_over_Position = df.pivot_table(values='Velocity', index='Lap_2', columns='2_cm_binned_position', aggfunc='mean')

# We plot the lap number against the averaged velocity per bin. The velocity is color-coded.
f, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(Velocity_over_Position, annot=False, ax=ax, cmap="coolwarm")

# The reward position is indicated by a red vertical line.
ax.vlines(284/2, 0, 64, colors='red')

### **Question**: Can we draw conclusions about the behavior of the animal?

## **Event triggered average**

### Load dataset

In [ ]:
neural_data = pd.read_excel('DataFrame_neural_data_event_triggered_avg.xlsx')

In [ ]:
spiking_data = pd.read_excel('DataFrame_spiking_data_event_triggered_avg.xlsx')

### **Task:** Check out the data.

In [ ]:
# Code goes here:


### Plot some examplary traces.

In [ ]:
counter = 0

plt.figure(figsize=(20, 10))

for cell in range(20):
    plt.plot(neural_data[cell] + counter)
    counter = counter + 1

### We further normalize the data by calculating the z-score for normalizing the data.

In [ ]:
# Calculate z-score from dFF (Optional)

raw_calcium = neural_data.T.values
#k = int(len(raw_calcium)/100*20)                    #uncomment if z-score is calculated from unnormalized data

z_score = []

for t in range(len(raw_calcium)):
    arr = raw_calcium[t]
    #F0 = arr.nsmallest(k).mean()
    std = arr.std()
    trace = arr/std                                  #if normalization is necessary: trace = (arr-F0)/std
    z_score.append(trace)

### In this recording we see the hippocampal CA1 region where PV+ Interneurons are optogenetically stimulated. We want to find out how the circuit responds to this.  

### **Task:** Go into the recording and retrieve the starting frames of each train of optogenetic stimulation.

In [ ]:
# Please enter the time stamps of each start frame:

stimulation_start = np.asarray([])


### **Solution:**

In [ ]:
stimulation_start = np.array([2187, 3432, 4335, 5191, 6518])

### For each neuron, we collect the individual responses to each optogenetic train. In total we have 5 trains of stimulation, so for each neuron we get 5 traces. 

In [ ]:
event_tuning_calcium_total = []   # Final list calcium
event_tuning_spiking_total = []   # Final list spiking
stim_idx = stimulation_start   # Stimulation marks
n_data = z_score   # Neural data
s_data = spiking_data.T.values   # Spiking data


for neuron in range(len(n_data)):   # We loop through all cells in the dataset
    event_tuning_neuron = []   # We collect the 5 calcium response traces for each neuron in this list
    spiking_neuron = []   # Same for the spiking data
    for e in range(len(stim_idx)):   # We loop through the stimulation marks
        event_tuning_neuron.append(n_data[neuron][stim_idx[e]-160:stim_idx[e]+480])   # We retrieve a time window of -5 sec to +10 sec. In total 20 sec (or 640 frames)
        spiking_neuron.append(s_data[neuron][stim_idx[e]-160:stim_idx[e]+480])      # Same for the spiking data
    event_tuning_calcium_total.append(event_tuning_neuron)   # We append the 5 traces collected for the individual neuron to the final list
    event_tuning_spiking_total.append(spiking_neuron)   # Same for the spiking data

### Plot the 5 traces for an exemplary neuron. The red lines indicate the time window of stimulation.

In [ ]:
neuron = 233

sns.heatmap(event_tuning_calcium_total[neuron], cmap='jet')
plt.vlines(x=160, ymin=5, ymax=0, color='red')
plt.vlines(x=320, ymin=5, ymax=0, color='red')

In [ ]:
neuron = 233

sns.heatmap(event_tuning_spiking_total[neuron], cmap='jet')
plt.vlines(x=160, ymin=5, ymax=0, color='red')
plt.vlines(x=320, ymin=5, ymax=0, color='red')

### From this we now can calculate the average response for each neuron.

In [ ]:
# Collect all averaged trial traces

avg_trials = []
avg_trials_spiking = []

for cell in range(np.asarray(event_tuning_calcium_total).shape[0]):
    avg_trials.append(np.mean(np.asarray(event_tuning_calcium_total)[cell], axis=0))
    avg_trials_spiking.append(np.mean(np.asarray(event_tuning_spiking_total)[cell], axis=0))

### Plot all average traces.

In [ ]:
plt.figure(figsize=(12,20));

sns.heatmap(np.asarray(avg_trials)[:, 0:], cmap='jet', vmin=0.01)
plt.vlines(x=160, ymin=810, ymax=0, color='red')
plt.vlines(x=320, ymin=810, ymax=0, color='red')

### **Question:** What do you see?

## **Dimensionality reduction**

### Next we prepare the data, so that we can perform principal component anaylsis (PCA). First we extract the part of the recording that we want to use for the PCA.

### **Task:** What part of the recording would make most sense? Enter the time points you want to use below.

In [ ]:
# Enter starting frame and ending frame

start_frame = 
end_frame = 


In [ ]:
neural_data_postStim = np.asarray(avg_trials)[:, start_frame:end_frame].reshape(np.shape(np.asarray(avg_trials)[:, start_frame:end_frame])[0],np.shape(np.asarray(avg_trials)[:, start_frame:end_frame])[1])

### Perform PCA.

In [ ]:
# create the model
pca = decomposition.PCA(n_components=4)
# fit the model on training data
pca.fit(neural_data_postStim)
# transformation on 2D space
pca_neural_data = pca.transform(neural_data_postStim)

### Plot scatter plot of first 2 principal components (PCs). 

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111)
pcm = ax.scatter(pca_neural_data[:,0], pca_neural_data[:,1])
#fig.colorbar(pcm, ax=ax)
plt.xlabel('PC1')
plt.ylabel('PC2')

### **Question:** What do you see?

## **Clustering**

### Perform clustering on dimension-reduced data.

### **Question:** Where would you expect the clusters?

In [ ]:
x = pca_neural_data

kmeans_euclidean = KMeansEuclidean(2)
y_kmeans_euclidean = kmeans_euclidean.fit(x).predict(x) # assign each smaple to a cluster

#plot results
plt.figure(figsize=(12,12))
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.scatter(x[:, 0], x[:, 1], c=y_kmeans_euclidean, s=10, cmap='brg', alpha=0.5);

### Plot average traces separated according to their cluster identity.

In [ ]:
heatmap_InhEx_Traces = np.concatenate((np.asarray(avg_trials)[np.asarray(y_kmeans_euclidean) == 0][:, :], np.asarray(avg_trials)[np.asarray(y_kmeans_euclidean) == 1][:, :]))

fig = plt.figure(figsize=(6, 16))

gs = gridspec.GridSpec(16, 2, figure=fig)
ax1 = fig.add_subplot(gs[:14, :])
ax1 = sns.heatmap(heatmap_InhEx_Traces, vmin=0.4, cmap='jet', cbar=False)

ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,
    left=False,       # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelleft=False) # labels along the bottom edge are off


plt.vlines(x=160, ymin=810, ymax=0, color='yellow')
plt.vlines(x=320, ymin=810, ymax=0, color='yellow')

ax2 = fig.add_subplot(gs[14:, :])
ax2.plot(np.mean(np.asarray(avg_trials)[np.asarray(y_kmeans_euclidean) == 1][:,:], axis=0), color='green');
ax2.plot(np.mean(np.asarray(avg_trials)[np.asarray(y_kmeans_euclidean) == 0][:,:], axis=0), color='blue');
ax2.margins(x=0)
ax2.axis('off')

### There is a lot of noise due to the optogenetic stimulation. We can plot the inferred spiking to see the cleared response.

In [ ]:
heatmap_InhEx_Traces = np.concatenate((np.asarray(avg_trials_spiking)[np.asarray(y_kmeans_euclidean) == 0][:, :], np.asarray(avg_trials_spiking)[np.asarray(y_kmeans_euclidean) == 1][:, :]))

fig = plt.figure(figsize=(6, 16))

gs = gridspec.GridSpec(16, 2, figure=fig)
ax1 = fig.add_subplot(gs[:14, :])
ax1 = sns.heatmap(heatmap_InhEx_Traces, cmap='jet', cbar=False)

ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,
    left=False,       # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelleft=False) # labels along the bottom edge are off


plt.vlines(x=160, ymin=810, ymax=0, color='yellow')
plt.vlines(x=320, ymin=810, ymax=0, color='yellow')

ax2 = fig.add_subplot(gs[14:, :])
ax2.plot(np.mean(np.asarray(avg_trials_spiking)[np.asarray(y_kmeans_euclidean) == 1][:,:], axis=0), color='green');
ax2.plot(np.mean(np.asarray(avg_trials_spiking)[np.asarray(y_kmeans_euclidean) == 0][:,:], axis=0), color='blue');
ax2.margins(x=0)
ax2.axis('off')

In [ ]:
heatmap_InhEx_Traces_BLPopulationActivity = np.concatenate((np.asarray(n_data)[np.asarray(y_kmeans_euclidean) == 0][:, :975], np.asarray(n_data)[np.asarray(y_kmeans_euclidean) == 1][:, :975]))

In [ ]:
fig = plt.figure(figsize=(6, 16))

gs = gridspec.GridSpec(16, 2, figure=fig)
ax1 = fig.add_subplot(gs[:14, :])
ax1 = sns.heatmap(heatmap_InhEx_Traces_BLPopulationActivity, vmin=0.01, cmap='jet', cbar=False)

ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,
    left=False,       # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelleft=False) # labels along the bottom edge are off


#ax1.vlines(x=160, ymin=661, ymax=0, color='yellow')
#ax1.vlines(x=320, ymin=661, ymax=0, color='yellow')

ax2 = fig.add_subplot(gs[14:, :])
ax2.plot(np.mean(np.asarray(n_data)[np.asarray(y_kmeans_euclidean) == 1][:,:975], axis=0), color='green');
ax2.plot(np.mean(np.asarray(n_data)[np.asarray(y_kmeans_euclidean) == 0][:,:975], axis=0), color='blue');
ax2.margins(x=0)
ax2.axis('off')

## **Spatial tuning**

### **Task:** Check out both data sets (df_day1 and df_day2).

In [ ]:
# Code goes here

### Visualize running behavior

In [ ]:
# Define which data frame is used - in this case we use the data frame that contains the position information
df = df_day1 

# We aligne the data - in this case we calculate the average velocity for each 2 cm bin
Velocity_over_Position = df.pivot_table(values='Velocity', index='Lap_2', columns='2_cm_binned_position', aggfunc='mean')

# We plot the lap number against the averaged velocity per bin. The velocity is color-coded.
f, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(Velocity_over_Position, annot=False, ax=ax, cmap="coolwarm")

# The reward position is indicated by a red vertical line.
ax.vlines(284/2, 0, 64, colors='red')

In [ ]:
# Define which data frame is used - in this case we use the data frame that contains the position information
df = df_day2 

# We aligne the data - in this case we calculate the average velocity for each 2 cm bin
Velocity_over_Position = df.pivot_table(values='Velocity', index='Lap_2', columns='2_cm_binned_position', aggfunc='mean')

# We plot the lap number against the averaged velocity per bin. The velocity is color-coded.
f, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(Velocity_over_Position, annot=False, ax=ax, cmap="coolwarm")

# The reward position is indicated by a red vertical line.
ax.vlines(284/2, 0, 71, colors='red')

### Check out spatial tuning of cells

In [ ]:
# Define data frame
df = df_day2

# Define Cell 
Cell = 122

# Create figure
fig = plt.figure(figsize=(16, 6))
gs = gridspec.GridSpec(9, 16, figure=fig)

# Plot heatmap of averaged deltaF/F for each 2 cm bin
ax1 = fig.add_subplot(gs[:7, :8])
firing = df.pivot_table(Cell, 'Lap_2', '2_cm_binned_position')
binned_avg = df.pivot_table(values=Cell, columns='2_cm_binned_position', aggfunc='sum')
ax1 = sns.heatmap(firing, vmin=df[Cell].min(), vmax=df[Cell].max(), ax=ax1, cmap='jet');
ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,
    left=False,       # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelleft=False) # labels along the bottom edge are off


# Plot average over spatial bins
ax2 = fig.add_subplot(gs[8:, :8])
avg_flour = df.pivot_table(values=Cell, columns='2_cm_binned_position', aggfunc='mean')
sns.heatmap(avg_flour, annot=False, ax=ax2, vmin=np.min(avg_flour.values), vmax=np.max(avg_flour.values), cmap='jet');
ax2.axis('off')

#Plot circular plot
binned_mean = df.pivot_table(values=Cell, columns='2_cm_binned_position', aggfunc='mean')
radii = binned_mean.to_numpy()

ax3 = fig.add_subplot(gs[:8, 8:], polar=True)
N = 180
theta = np.arange(0.0, 2 * np.pi, 2 * np.pi / N)
#radii = np.arange(0, N)
width = 2*np.pi / N
bars = ax3.bar(theta, radii[0], width=width, bottom=0.0, color='red')

### This plotting is rather calculation power intensiv. The circular plot is not necessarily required for inspection. 

In [ ]:
# Define data frame
df = df_day2

# Define target cell
cell=23

#Plot heatmap
plot_heatmap(cell, df, save=False, save_path='U:/PostDoc/Teaching/Cell_279.png', cmap_c='jet')

### **Task:** Explore the data. What are your thoughts? Specifically concider following points:  
- **Is there a difference in representation between day 1 and day 2?**
- **Do you observe events that you can relate to concepts previously learned in the course?**

## **Information rate**  
Spiking of CA1 neurons obviously carries information about space. Therefore, in this chapter we are going to calculate the spatial information, which can be used as a metric for defining and quantifying place cell integrity. 
Following equation will be used:

$\sum \limits _{j=1} ^{n} p_{i} \frac{\lambda_{i}}{\lambda} log_{2} (\frac{\lambda_{i}}{\lambda})$

where λi the mean firing rate in the i-th bin, λ the overall mean firing rate and pi the probability 
of the animal being in the i-th bin (occupancy in the i-th bin/total recording time). Spatial 
information in bits/spike was obtained by dividing the information rate with the mean firing 
rate of the cell. (taken from: Hoydal et al, 2020).

### Helper function

In [ ]:
def calculate_information_rate(data_frame, cell):
    occ_time = data_frame.pivot_table(values='Time_counter', columns='2_cm_binned_position', aggfunc='sum')/np.sum(data_frame.pivot_table(values='Time_counter', columns='2_cm_binned_position', aggfunc='sum').values)
    bfr = data_frame.pivot_table(values=cell, columns='2_cm_binned_position', aggfunc='sum').values/(data_frame.pivot_table(values='Time_counter', columns='2_cm_binned_position', aggfunc='sum').values/30)
    mfr = data_frame[cell].sum()/data_frame['Time'].max()
    bits_per_spike = np.nansum(occ_time.values*(bfr/mfr)*np.log(bfr/mfr)/np.log(2))
    return bits_per_spike

def bootstrap_fragments_shuffled(df, cell, Samples, window_size):
    peaks = []
    for m in range(Samples):
        fl_trace = np.array_split(df[cell], window_size)
        shuf_trace = shuffle(fl_trace)
        shuf_trace = pd.concat(shuf_trace)
        shuffled_df = pd.DataFrame({'Cell': shuf_trace.values, 'Position_binned': df['Position_binned'].values})
        avg_flour_shuffled = shuffled_df.pivot_table(values='Cell', columns='Position_binned', aggfunc='mean')
        peak_value = avg_flour_shuffled.T.max().values[0]
        peaks.append(peak_value)
    return peaks

### Load example data

In [ ]:
# Dataset #2
import time

start = time.time()

df_spiking_day2 = pd.read_excel('DataFrame_example_spiking_day2.xlsx')

end = time.time()
print(end - start)

### Calculate spatial information for each cell

In [ ]:
df = df_spiking_day2.loc[df_spiking_day2['Velocity'] > 2]

spatial_information_results = []

for cell in range(698):
    cell_spatial_information = calculate_information_rate(df, cell)
    spatial_information_results.append(cell_spatial_information)

### **Task:** The cells can be sorted according to their spatial information value (ascending order). Check out the cells with low spatial information score and those with high spatial infromation score.

In [ ]:
np.argsort(spatial_information_results)

In [ ]:
print('Cell with highest score: Cell# {} - {}'.format(np.argsort(spatial_information_results)[-1], spatial_information_results[np.argsort(spatial_information_results)[-1]]))
print('Cell with lowest score: Cell# {} - {}'.format(np.argsort(spatial_information_results)[1], spatial_information_results[np.argsort(spatial_information_results)[1]]))

### Now we can visualize the spatial firing (spiking) for individual cells.

In [ ]:
# Define data frame
df = df_spiking_day2.loc[df_spiking_day2['Velocity'] > 2]

# Define Cell 
Cell = 449

# Create figure
fig = plt.figure(figsize=(16, 6))
gs = gridspec.GridSpec(9, 16, figure=fig)

# Plot heatmap of averaged deltaF/F for each 2 cm bin
ax1 = fig.add_subplot(gs[:7, :8])
firing = df.pivot_table(Cell, 'Lap_2', '2_cm_binned_position')
binned_avg = df.pivot_table(values=Cell, columns='2_cm_binned_position', aggfunc='sum')
ax1 = sns.heatmap(firing, vmin=df[Cell].min(), vmax=df[Cell].max(), ax=ax1, cmap='jet');
ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,
    left=False,       # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelleft=False) # labels along the bottom edge are off


# Plot average over spatial bins
ax2 = fig.add_subplot(gs[8:, :8])
avg_flour = df.pivot_table(values=Cell, columns='2_cm_binned_position', aggfunc='mean')
sns.heatmap(avg_flour, annot=False, ax=ax2, vmin=np.min(avg_flour.values), vmax=np.max(avg_flour.values), cmap='jet');
ax2.axis('off')

#Plot circular plot
binned_mean = df.pivot_table(values=Cell, columns='2_cm_binned_position', aggfunc='mean')
radii = binned_mean.to_numpy()

ax3 = fig.add_subplot(gs[:8, 8:], polar=True)
N = 180
theta = np.arange(0.0, 2 * np.pi, 2 * np.pi / N)
#radii = np.arange(0, N)
width = 2*np.pi / N
bars = ax3.bar(theta, radii[0], width=width, bottom=0.0, color='red')

#plt.savefig(save_path, format='eps')

### Compared to the z-scored version, this looks much more sparse.

In [ ]:
# Define data frame
df = df_day2

# Define Cell 
Cell = 380

# Create figure
fig = plt.figure(figsize=(16, 6))
gs = gridspec.GridSpec(9, 16, figure=fig)

# Plot heatmap of averaged deltaF/F for each 2 cm bin
ax1 = fig.add_subplot(gs[:7, :8])
firing = df.pivot_table(Cell, 'Lap_2', '2_cm_binned_position')
binned_avg = df.pivot_table(values=Cell, columns='2_cm_binned_position', aggfunc='sum')
ax1 = sns.heatmap(firing, vmin=df[Cell].min(), vmax=df[Cell].max(), ax=ax1, cmap='jet');
ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,
    left=False,       # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelleft=False) # labels along the bottom edge are off


# Plot average over spatial bins
ax2 = fig.add_subplot(gs[8:, :8])
avg_flour = df.pivot_table(values=Cell, columns='2_cm_binned_position', aggfunc='mean')
sns.heatmap(avg_flour, annot=False, ax=ax2, vmin=np.min(avg_flour.values), vmax=np.max(avg_flour.values), cmap='jet');
ax2.axis('off')

#Plot circular plot
binned_mean = df.pivot_table(values=Cell, columns='2_cm_binned_position', aggfunc='mean')
radii = binned_mean.to_numpy()

ax3 = fig.add_subplot(gs[:8, 8:], polar=True)
N = 180
theta = np.arange(0.0, 2 * np.pi, 2 * np.pi / N)
#radii = np.arange(0, N)
width = 2*np.pi / N
bars = ax3.bar(theta, radii[0], width=width, bottom=0.0, color='red')

### **Question:** Do you notice anything interesting?

In [ ]:
plt.figure(figsize=(16,12))

# Plot the dF/F
plt.plot(df_day2[380-1].values[18000:19000]*3, color='blue')

# For comparison, plot the inferred spiking in blue
plt.plot(df_spiking_day2[380].values[18000:19000]*32, color='orange')

## **Bootstrapping**

### "Bootstrapping statistics is a form of hypothesis testing that involves resampling a single data set to create a multitude of simulated samples. Those samples are used to calculate standard errors, confidence intervals and for hypothesis testing."

### We split the recording (velocity > 2 cm/sec) for an individual cell into random-sized (5 sec < 10 sec) snipplets and reassamble them in random order. From this we calculate the new peak firing rate and compare it to the original one. This we repeat 500 times.

In [ ]:
df = df_spiking_day2.loc[df_spiking_day2['Velocity'] > 2]
start = time.time()

frame_rate = 32
cell = 380
bst_test_2 = bootstrap_fragments_shuffled(df=df, cell=cell, Samples=500, window_size=int(len(df[cell])/randrange(frame_rate*5, frame_rate*10)))
plt.hist(bst_test_2, bins=300, range=(0,0.6))
plt.axvline(df.pivot_table(values=cell, columns='2_cm_binned_position', aggfunc='mean').T.max().values[0], color='red')

bst_result = bst_test_2 < df.pivot_table(values=cell, columns='2_cm_binned_position', aggfunc='mean').T.max().values[0]
print("Percentage of smaller peaks: {}%".format((np.count_nonzero(bst_result == True)/(len(bst_result)/100))))

end = time.time()
print(end - start)

### No we create a data frame that contains the peak value, peak position and mutual information score for each neuron in our recording.

In [ ]:
df = df_spiking_day2

pc_peaks = []
pc_peak_pos = []
pc_information_rate = []

for c in range(697):
    peak_value = df.pivot_table(values=c, columns='Position_binned', aggfunc='mean').T.max().values[0]
    peak_pos = df.pivot_table(values=c, columns='Position_binned', aggfunc='mean').T.idxmax().values[0]
    inf_rate = calculate_information_rate(df, c)
    #print(peak_pos)
    pc_peaks.append(peak_value)
    pc_peak_pos.append(peak_pos)
    pc_information_rate.append(inf_rate)

df_place_cell_evaluation = pd.DataFrame({'Peak value': pc_peaks, 'Peak position': pc_peak_pos, 'Information rate': pc_information_rate})

In [ ]:
df_place_cell_evaluation

### Next we perform the bootstrapping. This is commented-out, because it would take an hour to run. Please load the prepared data frame.

In [ ]:
#df = df_spiking_day2
#shuffled_peaks = []

#frame_rate = 32

#start = time.time()
#for c in range(698):
    #peak_values = bootstrap_fragments_shuffled(
        #df=df, cell=c, Samples=500, window_size=int(len(df[c])/randrange(frame_rate*5, frame_rate*10)))
    #mask = peak_values < df.pivot_table(values=c, columns='Position_binned', aggfunc='mean').T.max().values[0]
    #shuffled_peaks.append(np.count_nonzero(mask == True)/(len(mask)/100))
    
#end = time.time()
#print(end - start)
    
#df_place_cell_evaluation['Percentile of smaller peaks'] = shuffled_peaks

In [ ]:
#df_place_cell_evaluation.to_excel('U:/PostDoc/Teaching/place_cell_evaluation.xlsx', index=False)

### Load data frame.

In [ ]:
df_place_cell_evaluation = pd.read_excel(
    'Place_cell_evaluation.xlsx', engine='openpyxl')

In [ ]:
df_place_cell_evaluation

### Select only those neurons that have >95% fraction of smaller peaks.

In [ ]:
target_df = df_place_cell_evaluation
df_moving = df_spiking_day2[df_spiking_day2['Velocity'] > 2]

idxLst = []

for c in (np.asarray(target_df.loc[target_df['Percentile of smaller peaks'] > 95].index)):
    idxLst.append(np.argmax(df_moving.pivot_table(values=c, columns='5_cm_binned_position', aggfunc='mean')))
          
dic = {'Identified place cells' : np.asarray(target_df.loc[target_df['Percentile of smaller peaks'] > 95].index), 'Peak Idx' : idxLst, 'Inf rate' : np.asarray(target_df['Information rate'].loc[target_df['Percentile of smaller peaks'] > 95])}
df_place_cells = pd.DataFrame(data=dic)

In [ ]:
df_place_cells

### Sort those neurons according to their peak position.

In [ ]:
df_place_cell_sorted = df_place_cells.sort_values(by = 'Peak Idx', ascending=False)

In [ ]:
df_place_cell_sorted

### Plot ordered cells as heatmap.

In [ ]:
PlaceCellsSorted = []
PlaceCellsNormSorted = []

df_moving = df_spiking_day2.iloc[32:-32, :][df_spiking_day2.iloc[32:-32, :]['Velocity'] > 2]
df = df_place_cells.sort_values(by = 'Peak Idx', ascending=False, ignore_index=True)
count = 0

for c in df['Identified place cells'].values:
    s = df_moving.pivot_table(values=c, columns='2_cm_binned_position', aggfunc='mean').values[0]
    #print(s.max())
    scaler = MinMaxScaler()
    t = scaler.fit_transform(s.reshape(-1,1))
    #print(t.max())
    PlaceCellsSorted.append(s)
    PlaceCellsNormSorted.append(t.T[0])
    count = count + 1
    #print(count)
    
fig = plt.figure(figsize=(20,10))
sns.heatmap(PlaceCellsNormSorted, cmap='YlGnBu_r')   #YlGnBu_r

plt.vlines(48.4/2, 0, 325, linestyles='dashed', colors='white')
plt.vlines(103.2/2, 0, 325, linestyles='dashed', colors='yellow')

plt.vlines(164.8/2, 0, 325, linestyles='dashed', colors='white')
plt.vlines(222.4/2, 0, 325, linestyles='dashed', colors='yellow')

plt.vlines(284/2, 0, 325, linestyles='dashed', colors='red')
plt.vlines(346/2, 0, 325, linestyles='dashed', colors='yellow')

plt.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,
    left=False,       # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelleft=False) # labels along the bottom edge are off

#plt.savefig('U:/PostDoc/Workshops/CAJAL 2023/Place_cell_map.eps', format='eps')

### Plot the average population activity of those place cells.

In [ ]:
fig = plt.figure(figsize=(16, 8))

plt.plot(np.mean(PlaceCellsNormSorted, axis=0))

### **Question:** What can we conclude from this?

### Other representation (line-plot) of the ordered place cells (just because it looks nice ;)).

In [ ]:
counter = 0

fig = plt.figure(figsize=(16, 8))

for trace in PlaceCellsNormSorted:
    plt.plot(trace*15 + counter, color='grey', linewidth=0.4)
    counter = counter - 1